# STAPL-3D feature extraction demo

This notebook demonstrates the core components of the STAPL-3D feature extraction module.

If you did not follow the STAPL-3D README: please find STAPL-3D and the installation instructions [here](https://github.com/RiosGroup/STAPL3D) before doing this demo.


First, define where you have put the data. Please change *datadir* to point to the *HFK16w* directory that you have unzipped.

In [ ]:
import os

datadir = './HFK16w'
dataset = 'HFK16w'
filestem = os.path.join(datadir, dataset)


Here we define the blocksize, blockmargin and the image dimensions.

In [ ]:
from stapl3d import Image
from glob import glob

n_proc = 16

bs = 176  # blocksize
bm = 64  # blockmargin

ipf = ''
blockdir = os.path.join(datadir, 'blocks')
filelist = glob(os.path.join(blockdir, '{}_*{}.h5'.format(dataset, ipf)))
filelist.sort()

image_in = '{}_bfc_block.ims'.format(filestem)
im = Image(image_in)
im.load(load_data=False)
dims = im.dims
im.close()


The STAPL-3D feature extraction module offers fast extraction of features from large amounts of data. We create a feature table for each datablock using parallel processing, then combine these feature tables while filtering out doubles of the segments that are represented in multiple datablocks.

In [ ]:
segs = [
    'segm/labels_memb_del_relabeled_fix_full', 
    'segm/labels_memb_del_relabeled_fix_memb',
    'segm/labels_memb_del_relabeled_fix_nucl',
]

featdir = os.path.join(datadir, 'profiling', 'features')
os.makedirs(featdir, exist_ok=True)

#seg_paths = [os.path.join(datadir, '{}.h5/{}'.format(dataset, seg)) for seg in segs]
seg_names = ['full', 'memb', 'nucl']
data_paths = [os.path.join(datadir, '{}_bfc_block.ims'.format(dataset))]
data_names = ['ch{:02d}'.format(ch) for ch in range(8)]
aux_data_path = []
downsample_factors = [1, 1, 1],
outputstem = os.path.join(featdir, dataset)
blocksize = [dims[0], bs, bs, dims[3], dims[4]]
blockmargin = [0, bm, bm, 0, 0]
blockrange = []
channels = []
filter_borderlabels = True
min_labelsize = 50
split_features = True
fset_morph = 'minimal'
fset_intens = 'minimal'
fset_addit = []


In [ ]:
from stapl3d.segmentation import features

parameter_file = '{}.yml'.format(filestem)
seg_names = ['full', 'memb', 'nucl']
seg_paths = ['{}.h5/segm/labels_memb_del_relabeled_fix_{}'.format(filestem, ids) for ids in seg_names]
features.estimate(image_in, parameter_file, seg_paths=seg_paths)


This generates 3 csv files for each datablock, in which the features for the full segment, and the nuclear and membranal parts are written to files in *HFK16w/profiling/features*. Finally, we merge all csv's together, taking the intensity features from the subsegments and retain the most interesting morphological and spatial features. We write then to a single outputfile as *HFK16w/profiling/features/HFK16w_features.csv*.

`features.postproc(image_in, parameter_file, seg_paths=seg_paths)`

In [ ]:
from stapl3d.segmentation.features import postprocess_features

postprocess_features(
    seg_paths,
    blocksize,
    blockmargin,
    blockrange=[],
    csv_dir=blockdir,
    csv_stem='{}_bfc_block'.format(dataset),
    feat_pf='_features',
    segm_pfs=['full', 'memb', 'nucl'],
    min_size_nucl=min_labelsize,
    save_border_labels=True,
    split_features=True,
    fset_morph=fset_morph,
    fset_intens=fset_intens,
    outputdir=datadir,
)
